In [ ]:
!pip install statsmodels
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_percentage_error

data = pd.read_csv('2023-2024-Horana-cleaned.csv') 
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()

daily_sales = data.groupby(['DATE', 'SalesPersonCode'])['NetAmount'].sum().reset_index()

def evaluate_model_daily(sales_code):
    person_data = daily_sales[daily_sales['SalesPersonCode'] == sales_code]
    person_data = person_data.set_index('DATE').asfreq('D').fillna(0)  

    train = person_data.iloc[:-7]  
    test = person_data.iloc[-7:]   

    try:
        model = ARIMA(train['NetAmount'], order=(5, 1, 0))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=7)

        results_df = pd.DataFrame({
            'DATE': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D'),
            'Predicted NetAmount': forecast.values
        })

        mape = mean_absolute_percentage_error(test['NetAmount'], forecast) * 100
        print(f"Model Evaluation (ARIMA) for {sales_code}: MAPE: {mape:.2f}%")

        if mape > 40:
            print(f"High MAPE ({mape:.2f}%). Trying ETS model for {sales_code}.")
            return evaluate_ets_daily(sales_code, test)

        return results_df

    except Exception as e:
        print(f"ARIMA failed for {sales_code} due to {e}. Switching to ETS...")
        return evaluate_ets_daily(sales_code, test)

def evaluate_ets_daily(sales_code, test):
    person_data = daily_sales[daily_sales['SalesPersonCode'] == sales_code]
    
    if len(person_data) < 10:
        print(f"Not enough data for {sales_code}, using mean-based prediction.")
        mean_forecast = [person_data['NetAmount'].mean()] * 7
        return pd.DataFrame({'DATE': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D'),
                             'Predicted NetAmount': mean_forecast})

    train = person_data.iloc[:-7]

    model = ExponentialSmoothing(train['NetAmount'], trend="add", seasonal=None)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=7)

    results_df = pd.DataFrame({
        'DATE': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D'),
        'Predicted NetAmount': forecast.values
    })

    mape = mean_absolute_percentage_error(test['NetAmount'], forecast) * 100
    print(f"Model Evaluation (ETS) for {sales_code}: MAPE: {mape:.2f}%")

    return results_df

sales_code = "254"  
forecast_results = evaluate_model_daily(sales_code)
print(forecast_results)


In [ ]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_percentage_error

data = pd.read_csv('2023-2024-Horana-cleaned.csv') 
data['DATE'] = pd.to_datetime(data['DATE'])
data['NetAmount'] = data['NetAmount'].abs()

daily_sales = data.groupby(['DATE', 'SalesPersonCode'])['NetAmount'].sum().reset_index()

def evaluate_model_daily(sales_code):
    person_data = daily_sales[daily_sales['SalesPersonCode'] == sales_code]
    person_data = person_data.set_index('DATE').asfreq('D').fillna(0)  # Ensure daily frequency

    train = person_data.iloc[:-7] 
    test = person_data.iloc[-7:]   

    try:
        model = ARIMA(train['NetAmount'], order=(5, 1, 0))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=7)

        results_df = pd.DataFrame({
            'DATE': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D'),
            'Predicted NetAmount': forecast.values
        })

        mape = mean_absolute_percentage_error(test['NetAmount'], forecast) * 100
        print(f"Model Evaluation (ARIMA) for {sales_code}: MAPE: {mape:.2f}%")

        if mape > 40:
            print(f"High MAPE ({mape:.2f}%). Trying ETS model for {sales_code}.")
            return evaluate_ets_daily(sales_code, test)

        generate_shap_report(model_fit, train, test)

        return results_df

    except Exception as e:
        print(f"ARIMA failed for {sales_code} due to {e}. Switching to ETS...")
        return evaluate_ets_daily(sales_code, test)

def evaluate_ets_daily(sales_code, test):
    person_data = daily_sales[daily_sales['SalesPersonCode'] == sales_code]
    
    if len(person_data) < 10:
        print(f"Not enough data for {sales_code}, using mean-based prediction.")
        mean_forecast = [person_data['NetAmount'].mean()] * 7
        return pd.DataFrame({'DATE': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D'),
                             'Predicted NetAmount': mean_forecast})

    train = person_data.iloc[:-7]

    model = ExponentialSmoothing(train['NetAmount'], trend="add", seasonal=None)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=7)

    results_df = pd.DataFrame({
        'DATE': pd.date_range(start=test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D'),
        'Predicted NetAmount': forecast.values
    })

    mape = mean_absolute_percentage_error(test['NetAmount'], forecast) * 100
    print(f"Model Evaluation (ETS) for {sales_code}: MAPE: {mape:.2f}%")

    return results_df

def generate_shap_report(model_fit, train, test):
    explainer = shap.Explainer(model_fit.predict, train['NetAmount'])
    shap_values = explainer(test['NetAmount'])

    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, show=False)
    plt.title("SHAP Report for ARIMA Model")
    plt.show()

sales_code = "265"  
forecast_results = evaluate_model_daily(sales_code)
print(forecast_results)


In [ ]:
import shap

explainer = shap.DeepExplainer(model, X_train)
shap_values = explainer.shap_values(X_test)

shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], X_test[0], feature_names=["NetAmount"])

shap.summary_plot(shap_values, X_test, feature_names=["NetAmount"])

shap.dependence_plot(0, shap_values[0], X_test, feature_names=["NetAmount"])